### first pass on API for ASE OOS

use cases:
- low/high degree nodes tend to distord spectral embedding, so omit from original embedding and then add back in with oos
- original graph too big, so make a subgraph and embed, then do oos for the rest
- have graph, embed it, get new graph defined on a larger set of nodes but with the same relationship between vertices. embed new graph with oos because it's generated by the same underlying distribution.

### notes
- add sklearn's exceptions.NotFittedError if we try to predict without fitting?
- for class-based sklearnish APIs, could add an augment_latent_positions toggle to predict if we want to have predicting update the latent positions matrix

In [1]:
import numpy as np

In [ ]:
# API 1: we just have an OOSEmbedder class
class OOSEmbed(BaseEmbed):
    def __init__(self, X):
        self.X = X
    
    def fit(self):
        """
        would take some input adjacency matrix self.X,
        embed it, and add self.pinv_, self.latent_right_,
        and self.singular_values_ to class attributes
        """
        # some code to make self.pinv
        pass


    def predict(self, y):
        """
        Given some latent position matrix X,
        its diagonal eigenvalue matrix D,
        and edge vector y,
        compute the least-squares estimate for the latent position of y.

        Parameters
        ----------
        X: array
            Latent position matrix
        eig_values: array
            Diagonal eigenvalue matrix
        y: array
            out-of-sample vectors to embed
        """
        return y @ self.pinv
        # maybe toggle for returning either 
        # *just* oos latent positions,
        # or an augmented ASE?
            

In [ ]:
from graspy.base import BaseEmbed

# API 2: we augment the AdjacencySpectralEmbed class with
# a predict method. This was Hayden's idea in the
# issue
class AdjacencySpectralEmbed(BaseEmbed):
    r"""
    bla bla bla
    """

    def __init__(
        self,
        n_components=None,
        n_elbows=2,
        algorithm="randomized",
        n_iter=5,
        check_lcc=True,
        diag_aug=True,
    ):
        super().__init__(
            n_components=n_components,
            n_elbows=n_elbows,
            algorithm=algorithm,
            n_iter=n_iter,
            check_lcc=check_lcc,
        )

        if not isinstance(diag_aug, bool):
            raise TypeError("`diag_aug` must be of type bool")
        self.diag_aug = diag_aug

    def fit(self, graph, y=None):
        # would add the pseudoinverse
        # as a class attribute in fit
        """
        bla bla bla
        """
        A = import_graph(graph)

        if self.check_lcc:
            if not is_fully_connected(A):
                msg = (
                    "Input graph is not fully connected. Results may not"
                    + "be optimal. You can compute the largest connected component by"
                    + "using ``graspy.utils.get_lcc``."
                )
                warnings.warn(msg, UserWarning)

        if self.diag_aug:
            A = augment_diagonal(A)

        self._reduce_dim(A)
        return self
    
    def _fit_transform(self):
        # would return embedded latent position
        # matrix, and add it as a class attribute.
        # overwrites the super() fit_transform
        pass
    
    def predict(self):
        # check that fit has been called first
        return y @ self.pinv

In [ ]:
# API 3: just have it as a simple function of some kind
# (I don't like this as much)
def oos_fit(X, y, eig_values):
    """
    Given some latent position matrix X,
    its diagonal eigenvalue matrix D,
    and edge vector y,
    compute the least-squares estimate for the latent position of y.
    """
    pinv = X @ np.diag(1/eig_values)
    return y @ pinv